Save an image of cumulative phase up to that point for each igram. Then combine to make a gif.

notebook make on 3-27-23.

In [1]:
# import stuff
import os,sys
import glob
from osgeo import gdal
import datetime 
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import imageio
%matplotlib  widget

Load in all unfilt, unwrapped igrams.

In [2]:
# igramsDir is where the cropped, unwrapped igrams are 
igramsDir = '/data/ocp4/LagunaSalada/cropped_9000_1800_18000_3600/cropped_northern_shore_LS_for_unwrapping/unfilt_unwrapped/'
# this one is to pick out the dates 
datesDir = '/data/ocp4/LagunaSalada/cropped_9000_1800_18000_3600/'
# shape of unwrapped igrams
dx = 3000
dy = 750
# load in dates, make sure they are in order
dates = [os.path.basename(x) for x in glob.glob(datesDir+"SLC_vh/2*")]; dates = sorted(dates)
nd = len(dates); numIgrams = nd-1
# set the driver first, only do once.
driver=gdal.GetDriverByName('ISCE')
unwr_full_res = np.ndarray((numIgrams,dy,dx),dtype='float')
# loop through the dates 
for i in range(numIgrams): # numIgrams if doing all of them at once
    date1=dates[i]
    date2=dates[i+1]
    igramFile = igramsDir+date1+'_'+date2+'_unfilt.unw'
    ds = gdal.Open(igramFile,gdal.GA_ReadOnly)
    unwr_full_res[i] = ds.GetRasterBand(1).ReadAsArray(0,0,dx,dy)
print('unfiltered unwrapped igrams loaded.')

unfiltered unwrapped igrams loaded.


Load the reference pixel mask and reference each igram. 

In [3]:
# load reference pixels mask 
maskFile = '/data/ocp4/LagunaSalada/cropped_9000_1800_18000_3600/cropped_northern_shore_LS_for_unwrapping/masks/ref_pixels_mask.r4'
ds = gdal.Open(maskFile,gdal.GA_ReadOnly)
ref_mask = ds.GetRasterBand(1).ReadAsArray(0,0,dx,dy)
print('Reference pixel mask loaded.')
# correct all igrams so there's a zero/stationary point  
mask_y,mask_x = np.nonzero(ref_mask)
unwr_full_res_cor = np.ndarray(np.shape(unwr_full_res),dtype='float') 
for i in range(numIgrams):
    # remove the average phase from small reference region at foot of mountains   
    unwr_full_res_cor[i] = unwr_full_res[i]-np.mean(unwr_full_res[i,mask_y,mask_x])
print('igrams corrected by subtracting avg value of ref pixels.')
# del unwr_full_res

Reference pixel mask loaded.
igrams corrected by subtracting avg value of ref pixels.


Calculate cumulative igrams.

In [4]:
# initialize the array
cum_unwr = np.ndarray(np.shape(unwr_full_res),dtype='float')
# calculate cumulative sum along the first axis of array 
cum_unwr = np.cumsum(unwr_full_res_cor,axis=0)
print('Cumulative phase calculated.')

# see what values to use for vmin and vmax to keep it constant in gif.
#print(max(cum_unwr.flatten())) # ~103.5
#print(min(cum_unwr.flatten())) # ~-108.9

Cumulative phase calculated.


Read in precip data.

In [5]:
# Read in 2day precipitation data
precip = np.loadtxt('/data/ocp4/LagunaSalada/3_precip/precip_2days.txt', dtype='float')
#precip = np.loadtxt('/data/ocp4/LagunaSalada/3_precip/precip_cumulative.txt', dtype='float')
print(np.shape(precip))

(211,)


Plot and save each image of each cumulative igram to be combined into a gif later. 

In [10]:
vmin = -20
vmax = 20
# Save these as isce-readable files so I don't have to run that each time 
saveDir = '/data/ocp4/LagunaSalada/11_unwrapped_igrams/igrams_for_gif_cumulative_disp/'
filenames = []

for i in range(numIgrams):  # for all, use range(numIgrams)
    date1 = dates[i]; date2 = dates[i+1]
    plt.figure(figsize=(10,9),tight_layout=True)
    plt.imshow(cum_unwr[i],aspect='auto',interpolation='nearest',vmin=vmin,vmax=vmax)
    plt.colorbar(orientation='vertical',pad=0.03,label='cum phase (rad)'); plt.set_cmap('jet')
    plt.xlim([dx,0]); plt.ylim([dy,0])
    plt.xticks([]); plt.yticks([]); #plt.xlabel('range'); plt.ylabel('azimuth')
    plt.title(str(date1)+', '+str(round(precip[i],2))+' cm precip\n'+str(date2)+', '+str(round(precip[i+1],2))+' cm precip'+'\n i = '+str(i))
    name_img = str(date1)+'-'+str(date2)+'_igram_'+str(i)+'_cum_unfilt_unwr.jpg'
    plt.savefig(saveDir+name_img, transparent=True)
    plt.close()
    filenames.append(name_img)
print('Figures saved.')

Figures saved.


If the files are already saved, make a file list to produce the gif in the next cell.

In [8]:
filenames = []
saveDir = '/data/ocp4/LagunaSalada/11_unwrapped_igrams/igrams_for_gif_cumulative_disp/'
for i in range(numIgrams):  # for all, use range(numIgrams)
    date1 = dates[i]; date2 = dates[i+1]
    name_img = str(date1)+'-'+str(date2)+'_igram_'+str(i)+'_cum_unfilt_unwr.jpg'
    filenames.append(name_img)
print('List of file names created.')

List of file names created.


In [9]:
# build gif
with imageio.get_writer('cum_disp_rads.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.v2.imread(saveDir+filename)
        writer.append_data(image)
print('Gif saved.')

Gif saved.
